**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
suppressMessages(suppressWarnings(source("../config/config_func.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(FD_RES, "source")
fglob = file.path(fdiry, "hic_intact*")
Sys.glob(fglob)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/hic_intact_K562_deep"       
[2] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/source/hic_intact_K562_ENCSR479XDG"

In [3]:
FOLDER="hic_intact_K562_deep"
fdiry = file.path(FD_RES, "source", FOLDER)
for (fname in dir(fdiry)) {print(fname)}

[1] "inter.hic"
[1] "localizedList_primary_10.bedpe"


## Import

In [4]:
fdiry = file.path(FD_RES, "source", FOLDER)
fname = "localizedList_primary_10.bedpe"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, col_names = FALSE, comment = "#")

dat_hic_loop = dat
print(dim(dat))
head(dat)

Rows: 128714 Columns: 46
── Column specification ──────────────────────────────────────────────────────
Delimiter: "\t"
chr  (7): X1, X4, X7, X8, X9, X10, X11
dbl (39): X2, X3, X5, X6, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 128714     46


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr10,72223000,72224000,chr10,72328000,72329000,.,.,.,.,⋯,72223010,72328480,28,0.006703467,3.735599,910,1320,180,480,0
chr10,64041000,64042000,chr10,67697000,67698000,.,.,.,.,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr10,78302000,78304000,chr10,79194000,79196000,.,.,.,.,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr10,13372000,13373000,chr10,13404000,13405000,.,.,.,.,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr10,132388000,132389000,chr10,132637000,132638000,.,.,.,.,⋯,132388100,132637710,6,0.029244716,2.577377,260,170,170,190,0
chr10,114550000,114552000,chr10,114876000,114878000,.,.,.,.,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


## Arrange

**Loop distance and size**

In [5]:
dat = dat_hic_loop
dat = dat[,1:6]

cnames = c("Chrom_A", "Start_A", "End_A", "Chrom_B", "Start_B", "End_B")
colnames(dat) = cnames

dat = dat %>%
    dplyr::mutate(
        Loop_A    = paste0(Chrom_A, ":", format(Start_A, scientific = FALSE), "-", format(End_A, scientific = FALSE)),
        Loop_B    = paste0(Chrom_B, ":", format(Start_B, scientific = FALSE), "-", format(End_B, scientific = FALSE))
    ) %>%
    dplyr::mutate(
        Loop_A = str_remove_all(Loop_A, " ")
    ) %>%
    dplyr::distinct()

dat_hic_loop_info = dat
print(dim(dat))
head(dat)

[1] 128714      8


Chrom_A,Start_A,End_A,Chrom_B,Start_B,End_B,Loop_A,Loop_B
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr10,72223000,72224000,chr10,72328000,72329000,chr10:72223000-72224000,chr10: 72328000- 72329000
chr10,64041000,64042000,chr10,67697000,67698000,chr10:64041000-64042000,chr10: 67697000- 67698000
chr10,78302000,78304000,chr10,79194000,79196000,chr10:78302000-78304000,chr10: 79194000- 79196000
chr10,13372000,13373000,chr10,13404000,13405000,chr10:13372000-13373000,chr10: 13404000- 13405000
chr10,132388000,132389000,chr10,132637000,132638000,chr10:132388000-132389000,chr10:132637000-132638000
chr10,114550000,114552000,chr10,114876000,114878000,chr10:114550000-114552000,chr10:114876000-114878000


In [6]:
dat = dat_hic_loop
dat = dat[,1:6]

cnames = c("Chrom_A", "Start_A", "End_A", "Chrom_B", "Start_B", "End_B")
colnames(dat) = cnames

dat = dat %>%
    dplyr::mutate(
        Loop_A = paste0(Chrom_A, ":", format(Start_A, scientific = FALSE), "-", format(End_A, scientific = FALSE)),
        Loop_B = paste0(Chrom_B, ":", format(Start_B, scientific = FALSE), "-", format(End_B, scientific = FALSE))
    ) %>%
    dplyr::mutate(
        Loop_A = str_remove_all(Loop_A, " "),
        Loop_B = str_remove_all(Loop_B, " ")
    ) %>%
    dplyr::mutate(
        Length_A = End_A - Start_A,
        Length_B = End_B - Start_B,
        Distance = abs((Start_A + End_A) / 2 - (Start_B + End_B) / 2),
        Loop     = paste0(Loop_A, "|", Loop_B)
    ) %>%
    dplyr::distinct()

dat_hic_loop_info = dat
print(dim(dat))
head(dat)

[1] 128714     12


Chrom_A,Start_A,End_A,Chrom_B,Start_B,End_B,Loop_A,Loop_B,Length_A,Length_B,Distance,Loop
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
chr10,72223000,72224000,chr10,72328000,72329000,chr10:72223000-72224000,chr10:72328000-72329000,1000,1000,105000,chr10:72223000-72224000|chr10:72328000-72329000
chr10,64041000,64042000,chr10,67697000,67698000,chr10:64041000-64042000,chr10:67697000-67698000,1000,1000,3656000,chr10:64041000-64042000|chr10:67697000-67698000
chr10,78302000,78304000,chr10,79194000,79196000,chr10:78302000-78304000,chr10:79194000-79196000,2000,2000,892000,chr10:78302000-78304000|chr10:79194000-79196000
chr10,13372000,13373000,chr10,13404000,13405000,chr10:13372000-13373000,chr10:13404000-13405000,1000,1000,32000,chr10:13372000-13373000|chr10:13404000-13405000
chr10,132388000,132389000,chr10,132637000,132638000,chr10:132388000-132389000,chr10:132637000-132638000,1000,1000,249000,chr10:132388000-132389000|chr10:132637000-132638000
chr10,114550000,114552000,chr10,114876000,114878000,chr10:114550000-114552000,chr10:114876000-114878000,2000,2000,326000,chr10:114550000-114552000|chr10:114876000-114878000


**Check results**

Make sure non of the number is converted to scientific notations

In [7]:
dat = dat_hic_loop_info
dat = dat %>% dplyr::filter(str_detect(Loop, "e"))
print(nrow(dat))
head(dat)

[1] 0


Chrom_A,Start_A,End_A,Chrom_B,Start_B,End_B,Loop_A,Loop_B,Length_A,Length_B,Distance,Loop
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>


## Save results for loop annotation

In [8]:
dat_loop_A = dat_hic_loop_info %>% dplyr::select(Chrom_A, Start_A, End_A) %>% dplyr::distinct() %>% dplyr::arrange(Chrom_A, Start_A, End_A)
dat_loop_B = dat_hic_loop_info %>% dplyr::select(Chrom_B, Start_B, End_B) %>% dplyr::distinct() %>% dplyr::arrange(Chrom_B, Start_B, End_B)

In [9]:
head(dat_loop_A)

Chrom_A,Start_A,End_A
<chr>,<dbl>,<dbl>
chr1,778000,779000
chr1,868000,870000
chr1,870000,872000
chr1,870000,875000
chr1,904000,905000
chr1,904000,906000


In [10]:
head(dat_loop_B)

Chrom_B,Start_B,End_B
<chr>,<dbl>,<dbl>
chr1,827000,828000
chr1,923000,924000
chr1,935000,940000
chr1,940000,941000
chr1,959000,960000
chr1,966000,968000


In [11]:
fdiry = file.path(FD_RES, "results", "region")
fglob = file.path(fdiry, "hic_intact*")
Sys.glob(fglob)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_deep"       
[2] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/hic_intact_ENCSR479XDG"

In [12]:
###
FOLDER="hic_intact_deep"
fdiry = file.path(FD_RES, "results", "region", FOLDER)

###
fname = "hic_intact.deep.Loop_A.bed.gz"
fpath = file.path(fdiry, fname)

dat = dat_loop_A
write_tsv(dat, fpath, col_names = FALSE)

###
fname = "hic_intact.deep.Loop_B.bed.gz"
fpath = file.path(fdiry, fname)

dat = dat_loop_B
write_tsv(dat, fpath, col_names = FALSE)

###
fname = "hic_intact.deep.Loop_info.tsv"
fpath = file.path(fdiry, fname)

dat = dat_hic_loop_info
write_tsv(dat, fpath)

```
###
FOLDER="hic_intact_deep"
fdiry = file.path(FD_RES, "results", "region", FOLDER, "loop")

###
fname = "loop_A.bed.gz"
fpath = file.path(fdiry, fname)

dat = dat_loop_A
write_tsv(dat, fpath, col_names = FALSE)

###
fname = "loop_B.bed.gz"
fpath = file.path(fdiry, fname)

dat = dat_loop_B
write_tsv(dat, fpath, col_names = FALSE)

###
fname = "loop_info.tsv"
fpath = file.path(fdiry, fname)

dat = dat_hic_loop_info
write_tsv(dat, fpath)
```